<div style='text-align: center;'>
<img src="../images/math60082-banner.png" alt="image" width="80%" height="auto">
</div>

# Lab Solutions - Week 2

# Tasks

1. Integrate $f(x)=\sin(x)$ and $f(x)=\cos(x)$ over the region $\left[0,\frac{3}{4}\pi\right]$. Verify the accuracy of your results.


In [1]:
from scipy.integrate import quad as QUAD
from math import sin,pi,cos 

lower_limit = 0.
upper_limit = 0.75*pi

print("Solve I_1 = int_0^{3/4 pi} sin(x) dx ")
I1 = QUAD( sin , lower_limit , upper_limit )
print("Numerical Approx := ",I1,"; Analytic Value := ", -cos(upper_limit) + cos(lower_limit))

print("Solve I_1 = int_0^{3/4 pi} cos(x) dx ")
I2 = QUAD( cos , lower_limit , upper_limit )
print("Numerical Approx := ",I2,"; Analytic Value := ", sin(upper_limit) - sin(lower_limit))


Solve I_1 = int_0^{3/4 pi} sin(x) dx 
Numerical Approx :=  (1.7071067811865472, 1.8952692539670438e-14) ; Analytic Value :=  1.7071067811865475
Solve I_1 = int_0^{3/4 pi} cos(x) dx 
Numerical Approx :=  (0.7071067811865476, 1.436323259749347e-14) ; Analytic Value :=  0.7071067811865476


2. Consider that you are required to integrate the function $f(x)=\max(x,e^{\frac{x}{2}}-1)$ over the region $[0,5]$. How might you best deal with this problem?

Note that this function is not smooth, so numerical approximations that assume smooth functions will introduce errors. We can deal with this by manually splitting the function up into piecewise smooth regions. So
$$
f(x) = 
\begin{cases}
x & \text{ if } x<\alpha \\
e^{\frac{x}{2}}-1 & \text{ if } x\geq \alpha
\end{cases}
$$
but we need to find $\alpha$. To find $\alpha$ we would set $g(x)=x-e^{\frac{x}{2}}+1$ and find the root $g(x)=0$ such that $x>0$.

In [2]:
from scipy.optimize import root_scalar as findRoot
from math import exp
alpha = findRoot( lambda x:x-exp(x/2.)+1 ,x0=2.,x1=5 )
alpha
print(" Solution of g(x)=0 is x=",alpha.root)

 Solution of g(x)=0 is x= 2.5128624172523284


Now split integration, so $I = I_1 + I_2$ where
$$
I_1 = \int_0^\alpha x dx
$$
and
$$
I_2 = \int_\alpha^5 e^{\frac{x}{2}} - 1 dx.
$$
to get:

In [3]:
I1 = QUAD( lambda x: x , 0., alpha.root )
I2 = QUAD( lambda x: exp(x/2.) -1 , alpha.root , 5. )
I = I1[0] + I2[0]
print(" Integral = ",I)

 Integral =  18.009364268174245


Note that once the integral is split, both sides could be calculated analytically.

3. Experiment with the lower and upper limits to see what effect they have. Can you propose what would be the _best_ values to choose in this case? Explain your reasoning.

In [4]:
# function to integrate normal distribution multiplied by 1/(1+t^2)
def Gx_integrate_test( x ):
    lower_limit=-1e16
    upper_limit=1e16
    if x<lower_limit:
        return 0.
    elif x>upper_limit:
        return Gx_integrate_test(upper_limit)
    from scipy.integrate import quad as QUAD
    from math import exp,pi,sqrt
    return (1./sqrt(2.*pi))*QUAD(lambda t: exp(-t*t/2.)/(1+t*t) , lower_limit, x)[0]



In [5]:
print("|", Gx_integrate_test(-2.0),"|", Gx_integrate_test(-1.0) ,"|",Gx_integrate_test(0.0),"|", Gx_integrate_test(1.0),"|", Gx_integrate_test(2.0),"|")

| 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |


Results with different values of the min/max limit in the function. Remember the limits are approximating $\pm\infty$ so they shouldn't be affecting the results. They should be chosen to be just large enough to stop affecting the results, and not too large so as to be wasting calculation time.

| Limits | G(-2) | G(-1) | G(0) | G(1) | G(2) |
|---|---|---|---|---|---|
| -1.0,1.0 | 0.0 | 0.0 | 0.27582626905934127 | 0.5516525381186825 | 0.5516525381186825 |
| -2.0,2.0 | 0.0 | 0.04846166762812421 | 0.3242879366874654 | 0.6001142057468069 | 0.6485758733749308 |
| -4.0,4.0 | 0.003550144298797249 | 0.05201181192692145 | 0.3278380809862627 | 0.6036643500456039 | 0.6521260176737282 |
| -8.0,8.0 | 0.0035518345219338033 | 0.05201350215005803 | 0.3278397712093992 | 0.6036660402687405 | 0.6521277078968646 |
| -16.0,16.0 |  0.003551834521933837 | 0.05201350215005802 | 0.3278397712093992 | 0.6036660402687405 | 0.6521277078968657 |
| -160,160 | 0.0035518345219338098 | 0.05201350215005802 | 0.3278397712093996 | 0.6036660402687405 | 0.6521277078968648 |
| -1e16,1e16 | 0.0 | 0.0 | 0.0 | 0.0 | 0.0 |

Values change a lot going from 1 to 2 to 4. We can see that values don't change much between 8 and 16, so a value of 8 (or perhaps 10?) would look to prove sufficient in this case. Choosing a huge number such as 1e16 which is ten to the power sixteen, actually gives the **wrong** result. When sampling this function over such a large interval it looks as though it would be zero everywhere.

In [6]:
# function to integrate normal distribution multiplied by 1/(1+t^2)
def Gx_integrate( x ):
    lower_limit=-10
    upper_limit=10
    if x<lower_limit:
        return 0.
    elif x>upper_limit:
        return Gx_integrate(upper_limit)
    from scipy.integrate import quad as QUAD
    from math import exp,pi,sqrt
    return (1./sqrt(2.*pi))*QUAD(lambda t: exp(-t*t/2.)/(1+t*t) , lower_limit, x)[0]


4. Try to write a function `Gxg_integrate( float: x , g )` which takes the `g` as an argument.

In [7]:
# function to integrate normal distribution multiplied by 1/(1+t^2)
def Gxg_integrate( x , g):
    lower_limit=-10
    upper_limit=10
    if x<lower_limit:
        return 0.
    elif x>upper_limit:
        return Gxg_integrate(upper_limit)
    from scipy.integrate import quad as QUAD
    from math import exp,pi,sqrt
    return (1./sqrt(2.*pi))*QUAD(lambda t: exp(-t*t/2.)*g(t) , lower_limit, x)[0]


5. Test the efficiency of calculation for $N(x)$ against the version from the special function module in `scipy`.

In [8]:
from timeit import timeit
from scipy.special import ndtr as ND 
# function to integrate cumulative normal distribution
def Nx_integrate( x ):
    if x<-15.0:
        return 0.
    elif x>15.0:
        return 1.0
    from scipy.integrate import quad as QUAD
    from math import exp,pi,sqrt
    return 0.5 + (1./sqrt(2.*pi))*QUAD(lambda t: exp(-t*t/2.), 0, x)[0]


In [9]:

n = 100000
script="ND(1.)"
timeIntegrate = timeit( script,number=n,globals=globals() )

print("Time taken to run ",n," calls to the function ",script, " is ", timeIntegrate," seconds.")

script="Nx_integrate(1.)"
timeIntegrate = timeit( script,number=n,globals=globals() )

print("Time taken to run ",n," calls to the function ",script, " is ", timeIntegrate," seconds.")

Time taken to run  100000  calls to the function  ND(1.)  is  0.07656527499784715  seconds.
Time taken to run  100000  calls to the function  Nx_integrate(1.)  is  1.0631936240024515  seconds.


6. Test the efficiency of calculating $G(x)$ when $g$ is written inside the function, versus when $g$ is passed in as an argument. Does the flexibility of the second method come at a cost?

First test our $G(x)$ function with $g$ written inside.

In [10]:
n = 100000
script="Gx_integrate(1.)"
timeIntegrate = timeit( script,number=n,globals=globals() )

print("Time taken to run ",n," calls to the function ",script, " is ", timeIntegrate," seconds.")

Time taken to run  100000  calls to the function  Gx_integrate(1.)  is  4.763800018998154  seconds.


This takes about 5 times longer than before. This is because the integral between $\infty$ and 0 is being calculated as part of the solution. You could provide a vast speed up here by precalculating it and using the formula $$G(x) = G(0) + \int_0^x ...$$

Now define a function `def g_func` that returns the appropriate value, and pass as argument into the function `Gx_integrate_g`.

In [11]:
n = 100000

def g_func(t):
    return 1./(1+t*t)

script="Gxg_integrate(1.,g_func)"
timeIntegrate = timeit( script,number=n,globals=globals() )

print("Time taken to run ",n," calls to the function ",script, " is ", timeIntegrate," seconds.")

Time taken to run  100000  calls to the function  Gxg_integrate(1.,g_func)  is  5.391431061998446  seconds.


We can see there is a slight time penalty (around 10%) introduced but our definition is more flexible and can be used again for other problems.

In [12]:
n = 100000
script="Gxg_integrate(1.,lambda t:1./(1+t*t))"
timeIntegrate = timeit( script,number=n,globals=globals() )

print("Time taken to run ",n," calls to the function ",script, " is ", timeIntegrate," seconds.")

Time taken to run  100000  calls to the function  Gxg_integrate(1.,lambda t:1./(1+t*t))  is  5.710484757000813  seconds.


There is not much difference when we do the same again with a `lambda` function.